In [18]:
import os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras import initializers
from keras import metrics
import matplotlib.pyplot as mpl
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [19]:
#Get the current working directory
cwd = os.getcwd()
print(cwd)

C:\Users\kayde\OneDrive\Desktop\mannada\RainfallData


In [20]:
#the variables as recorded by the station
variables = [
        'dswr',
        'lftx',
        'mslp',
        'p__f',
        'p__u',
        'p__v',
        'p__z',
        'p_th',
        'p_zh',
        'p5_f',
        'p5_u',
        'p5_v',
        'p5_z',
        'p5th',
        'p5zh',
        'p8_f',
        'p8_u',
        'p8_v',
        'p8_z',
        'p8th',
        'p8zh',
        'p500',
        'p850',
        'pottmp',
        'pr_wtr',
        'prec',
        'r500',
        'r850',
        'rhum',
        'shum',
        'temp',
]

#The three regions
regions = [
        '82.5', #A
        '85.0', #B
        '87.5', #C
]

'''
TAKING IN ALL THE X VARIABLES
'''

#define an empty dictionary
boxes = {}

#iterating over regions
for region in regions:
    
    df_li = pd.DataFrame() # empty dataframe
    
    #iterating over variables
    for var in variables:
        
        path = os.path.join(cwd,"DATA\\BOX_20N_%sE\\ncep_%s.dat"%(region,var)) # Cleverly curated path for automation
        
        with open(path) as file: #open the respected variable file for the region
            
            temp = file.read().splitlines()
            temp = [float(i) for i in temp]
            
        df_li[region + '_' + var] = temp #and for every variable, store that data under header: <coordinate_variable>
        
    boxes[region]=df_li #and assign that dataframe for every region
    
'''
TAKING IN ALL THE Y VALUES
'''

rain = []

for i in range(5): # iterate from 0 to 4 i.e. 5 times
    
    path = os.path.join(cwd,"DATA\\rain%d.dat"%(i+1)) # join cwd to path of rain data
    
    with open(path) as file:
        
        lines = file.read().splitlines()
        lines = [float(i) for i in lines]
        rain.append( lines )  #store in rain list

In [21]:
#time for some data refining and model training

#Trimming BOXES, this is to match with the rain data, day-by-day
for region in regions:
    boxes[region] = boxes[region].iloc[4749:,] # removing 4749 rows from front
    boxes[region] = boxes[region].iloc[:10957,] # keeping only 10957 of the rest

#sample take BOX A and PLACE 1
df = boxes['82.5']
rain1 = rain[0]

In [22]:
class predictionModel:
    '''
    This is a class defining the Prediction Neural Network, Its input filtering,
    '''
    def __init__(self,X,Y,classStr,N=1,K=31):
        if classStr == 'regressor':
            self.__model = self.__neuralNetworkRegressor(N,K)
        elif classStr == 'classifier':
            self.__model = self.__neuralNetworkClassifier(N,K)
        else:
            print('\nError: model %s not found'%(classStr))
        self.__K = K
        self.__N = N
        self.__xdata = X
        self.__ydata = Y
        self.__class = classStr
    def __neuralNetworkRegressor(self,N,K):
        model = Sequential(
            [
                Dense(
                    31*N,
                    input_dim=31*N,
                    activation='tanh',
                     #next layer is relu, so it needs normalization
                    kernel_initializer=initializers.GlorotNormal(seed=None), #glorot works good with sigmoid
                    bias_initializer='zeros',
                ),
                Dense(
                    K,
                    activation='relu', #relu, for better regression
                    kernel_initializer=initializers.HeNormal(seed=None), #He Normal goes good with relu
                    bias_initializer='zeros',
                ),
                Dense(
                    1,
                    kernel_initializer='normal',
                    bias_initializer='zeros',
                ),
            ]
        )
        model.compile(loss='mean_squared_error',optimizer='adam',metrics=[metrics.MeanAbsoluteError()])
        return model
    
    def __neuralNetworkClassifier(self,N,K):
        
        model = Sequential(
            [
                Dense(
                    31*N,
                    input_dim=31*N,
                    activation='sigmoid',
                    kernel_initializer=initializers.HeNormal(seed=None), #He Normal goes good with relu
                    bias_initializer='zeros',
                ),
                Dense(
                    K,
                    activation='relu', 
                    kernel_initializer=initializers.HeNormal(seed=None), #He Normal goes good with relu
                    bias_initializer='zeros',
                ),
                Dense(
                    1,
                    activation='sigmoid', # heard its the best activation for classfication output
                    kernel_initializer='normal',
                    bias_initializer='zeros',
                ),
            ]
        )
        model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model
    
    def trainNetwork(self):
        xtrain, xtest, ytrain, ytest = train_test_split(self.__xdata,self.__ydata,test_size=0.2)#splitting test-train
        print("Initiating Training Sequence\n")
        history = self.__model.fit(
            xtrain,
            ytrain,
            epochs = 10,
            batch_size = 5,
        )
        print("\n\nInitiating Testing Sequence\n")
        metrics = self.__model.evaluate(
            xtest,
            ytest,
            batch_size = 5,
        )
    def evaluateNetwork(self):
        '''
        This Function isn't really working now,
        Will come back later
        '''
        print("\nInitiating Evaluation Sequence\n")
        evaluators=[
            ('standardize',StandardScaler()),
            ('mlp',KerasRegressor(
                build_fn =  self.__neuralNetwork(self.__N,self.__K),
                epochs = 10,
                batch_size = 5,
                verbose = 0,
            )),
        ]
        pipeline = Pipeline(evaluators)
        results = cross_val_score(
            pipeline,
            self.__xdata,
            self.__ydata,
            cv = KFold(n_splits=10)
        )
        print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))
        

In [23]:
ydat = rain1.copy()
xdat = df.values.tolist()
rainFallPredictor = predictionModel(xdat,ydat,'regressor',1,10)
rainFallPredictor.trainNetwork()

Initiating Training Sequence

Epoch 1/10
1753/1753 [==============================] - 2s 1ms/step - loss: 64.4106 - mean_absolute_error: 3.4863
Epoch 2/10
1753/1753 [==============================] - 2s 1ms/step - loss: 58.5443 - mean_absolute_error: 3.1952
Epoch 3/10
1753/1753 [==============================] - 2s 1ms/step - loss: 58.3774 - mean_absolute_error: 3.1052
Epoch 4/10
1753/1753 [==============================] - 2s 1ms/step - loss: 58.2241 - mean_absolute_error: 3.0803
Epoch 5/10
1753/1753 [==============================] - 2s 1ms/step - loss: 57.7661 - mean_absolute_error: 3.0958
Epoch 6/10
1753/1753 [==============================] - 2s 1ms/step - loss: 57.2289 - mean_absolute_error: 3.1051
Epoch 7/10
1753/1753 [==============================] - 2s 1ms/step - loss: 57.6862 - mean_absolute_error: 3.1006
Epoch 8/10
1753/1753 [==============================] - 2s 1ms/step - loss: 57.3614 - mean_absolute_error: 3.0553
Epoch 9/10
1753/1753 [==============================] - 2s

In [24]:
df_2 = df.copy()
df_2['rain'] = rain1
df_2.drop(df_2[df_2['rain'] < 2.5].index, inplace = True)
ydata_2 = df_2['rain'].values.tolist()
df_2 = df_2.drop(['rain'],axis=1)
xdata_2=df_2.values.tolist()
rainFallPredictor2 = predictionModel(xdata_2,ydata_2,'regressor',1,400)
rainFallPredictor2.trainNetwork()

Initiating Training Sequence

Epoch 1/10
326/326 [==============================] - 1s 2ms/step - loss: 250.1431 - mean_absolute_error: 10.0545
Epoch 2/10
326/326 [==============================] - 1s 2ms/step - loss: 231.3875 - mean_absolute_error: 10.0386
Epoch 3/10
326/326 [==============================] - 1s 2ms/step - loss: 229.8502 - mean_absolute_error: 10.0250
Epoch 4/10
326/326 [==============================] - 1s 2ms/step - loss: 229.3216 - mean_absolute_error: 10.0191
Epoch 5/10
326/326 [==============================] - 1s 2ms/step - loss: 227.3964 - mean_absolute_error: 10.0135
Epoch 6/10
326/326 [==============================] - 1s 2ms/step - loss: 227.5672 - mean_absolute_error: 9.9436
Epoch 7/10
326/326 [==============================] - 1s 2ms/step - loss: 227.5914 - mean_absolute_error: 9.9406
Epoch 8/10
326/326 [==============================] - 1s 2ms/step - loss: 225.8918 - mean_absolute_error: 9.9213
Epoch 9/10
326/326 [==============================] - 1s 2ms/

In [25]:
ydata_3 = rain1
for i in range(len(ydata_3)):
    if ydata_3[i] > 2.5:
        ydata_3[i] = 1;
    else:
        ydata_3[i] = 0;
xdata_3 = df.values.tolist()
rainFallPredictor3 = predictionModel(xdata_3,ydata_3,'classifier',1,200)
rainFallPredictor3.trainNetwork()

Initiating Training Sequence

Epoch 1/10
1753/1753 [==============================] - 2s 1ms/step - loss: 0.3690 - accuracy: 0.8410
Epoch 2/10
1753/1753 [==============================] - 3s 1ms/step - loss: 0.3389 - accuracy: 0.8510
Epoch 3/10
1753/1753 [==============================] - 3s 1ms/step - loss: 0.3325 - accuracy: 0.8541
Epoch 4/10
1753/1753 [==============================] - 3s 2ms/step - loss: 0.3238 - accuracy: 0.8575
Epoch 5/10
1753/1753 [==============================] - 3s 2ms/step - loss: 0.3202 - accuracy: 0.8570
Epoch 6/10
1753/1753 [==============================] - 3s 2ms/step - loss: 0.3155 - accuracy: 0.8577
Epoch 7/10
1753/1753 [==============================] - 3s 1ms/step - loss: 0.3158 - accuracy: 0.8597
Epoch 8/10
1753/1753 [==============================] - 3s 2ms/step - loss: 0.3111 - accuracy: 0.8593
Epoch 9/10
1753/1753 [==============================] - 3s 2ms/step - loss: 0.3076 - accuracy: 0.8610
Epoch 10/10
1753/1753 [=============================